# BERT (Updated 1 Feb 2025, Available CUDA)

We shall implement BERT.  For this tutorial, you may want to first look at my Transformers tutorial to get a basic understanding of Transformers. 

For BERT, the main difference is on how we process the datasets, i.e., masking.   Aside from that, the backbone model is still the Transformers.

In [1]:
import os

# show all 4 GPUs
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

os.environ["http_proxy"]  = "http://192.41.170.23:3128"
os.environ["https_proxy"] = "http://192.41.170.23:3128"

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)
print("cuda available:", torch.cuda.is_available())
print("visible gpu count:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))

SEED = 1234
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


device: cuda
cuda available: True
visible gpu count: 1
0 NVIDIA GeForce RTX 2080 Ti


In [2]:
import math
import re
from   random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os

## 1. Data

For simplicity, we shall use very simple data like this.

In [3]:
from datasets import load_dataset

# Load BookCorpus dataset
# Source: https://huggingface.co/datasets/bookcorpus/bookcorpus
# Credit: Yukun Zhu et al., "Aligning Books and Movies: Towards Story-like 
# Visual Explanations by Watching Movies and Reading Books", arXiv 2015.
# We use only the first 100,000 samples for this assignment.
dataset = load_dataset('bookcorpus', split='train[:100000]')
dataset

Dataset({
    features: ['text'],
    num_rows: 100000
})

In [4]:
sentences = dataset['text']
text = [x.lower() for x in sentences] #lower case
text = [re.sub("[.,!?\\\\-]", '', x) for x in text] #clean all symbols


In [5]:
for sentence in text:
    print(sentence, "_____")
    words = sentence.split()
    print(words)
    break

usually  he would be tearing around the living room  playing with his toys  _____
['usually', 'he', 'would', 'be', 'tearing', 'around', 'the', 'living', 'room', 'playing', 'with', 'his', 'toys']


### Making vocabs

Before making the vocabs, let's remove all question marks and perios, etc, then turn everything to lowercase, and then simply split the text. 

In [6]:
from tqdm.auto import tqdm

# Combine everything into one to make vocab
word_list = list(set(" ".join(text).split()))
word2id = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}  # special tokens

# Create the word2id in a single pass
for i, w in tqdm(enumerate(word_list), desc="Creating word2id"):
    word2id[w] = i + 4  # because 0-3 are already occupied

# Precompute the id2word mapping (this can be done once after word2id is fully populated)
id2word = {v: k for k, v in word2id.items()}
vocab_size = len(word2id)
vocab_size

Creating word2id: 0it [00:00, ?it/s]

23068

In [7]:
vocab_size = len(word2id)

# List of all tokens for the whole text
token_list = []

# Process sentences more efficiently
for sentence in tqdm(text, desc="Processing sentences"):
    token_list.append([word2id[word] for word in sentence.split()])

# Now token_list contains the tokenized sentences

Processing sentences:   0%|          | 0/100000 [00:00<?, ?it/s]

In [8]:
#take a look at sentences
sentences[:2]

['usually , he would be tearing around the living room , playing with his toys .',
 'but just one look at a minion sent him practically catatonic .']

In [9]:
#take a look at token_list
token_list[:2]

[[20340,
  22586,
  20861,
  5432,
  268,
  11215,
  16619,
  10544,
  14510,
  22209,
  5372,
  17596,
  22884],
 [8726, 11932, 18491, 4484, 7756, 12391, 19870, 12187, 13805, 14041, 381]]

In [10]:
#testing one sentence
for tokens in token_list[0]:
    print(id2word[tokens])

usually
he
would
be
tearing
around
the
living
room
playing
with
his
toys


## 2. Data loader

We gonna make dataloader.  Inside here, we need to make two types of embeddings: **token embedding** and **segment embedding**

1. **Token embedding** - Given "The cat is walking. The dog is barking", we add [CLS] and [SEP] >> "[CLS] the cat is walking [SEP] the dog is barking". 

2. **Segment embedding**
A segment embedding separates two sentences, i.e., [0 0 0 0 1 1 1 1 ]

3. **Masking**
As mentioned in the original paper, BERT randomly assigns masks to 15% of the sequence. In this 15%, 80% is replaced with masks, while 10% is replaced with random tokens, and the rest 10% is left as is.  Here we specified `max_pred` 

4. **Padding**
Once we mask, we will add padding. For simplicity, here we padded until some specified `max_len`. 

Note:  `positive` and `negative` are just simply counts to keep track of the batch size.  `positive` refers to two sentences that are really next to one another.

In [11]:
batch_size = 16       # increased from 6 for better training
max_mask   = 10       # max masked tokens when 15% exceed, it will only be max_pred
max_len    = 128      # reduced from 1000; most BookCorpus sentences are short, so 128 is sufficient

In [12]:
def make_batch():
    batch = []
    positive = negative = 0  #count of batch size; we want half positive pairs (next sentence) and half negative
    while positive != batch_size/2 or negative != batch_size/2:
        
        #randomly choose two sentence indices
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))
        #retrieve the two sentences
        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]

        #1. token embedding - append CLS and SEP
        input_ids = [word2id['[CLS]']] + tokens_a + [word2id['[SEP]']] + tokens_b + [word2id['[SEP]']]
        
        # Truncate if longer than max_len to avoid index errors with positional embedding
        if len(input_ids) > max_len:
            input_ids = input_ids[:max_len]

        #2. segment embedding - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)
        segment_ids = segment_ids[:max_len]  # also truncate segment_ids to match

        #3. mask language modeling
        #masked 15%, but should be at least 1 but does not exceed max_mask
        n_pred =  min(max_mask, max(1, int(round(len(input_ids) * 0.15))))
        #get the pos that excludes CLS and SEP and shuffle them
        cand_maked_pos = [i for i, token in enumerate(input_ids) if token != word2id['[CLS]'] and token != word2id['[SEP]']]
        shuffle(cand_maked_pos)
        masked_tokens, masked_pos = [], []
        #simply loop and change the input_ids to [MASK]
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)  #remember the position
            masked_tokens.append(input_ids[pos]) #remember the tokens
            #80% replace with a [MASK], but 10% will replace with a random token
            if random() < 0.1:  # 10%
                index = randint(0, vocab_size - 1) # random index in vocabulary
                input_ids[pos] = word2id[id2word[index]] # replace
            elif random() < 0.9:  # 80%
                input_ids[pos] = word2id['[MASK]'] # make mask
            else:  #10% do nothing
                pass

        # pad the input_ids and segment ids until the max len
        n_pad = max_len - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # pad the masked_tokens and masked_pos to make sure the length is max_mask
        if max_mask > n_pred:
            n_pad = max_mask - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        #check if first sentence really comes before the second sentence
        #also make sure positive is exactly half the batch size
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
            negative += 1
            
    return batch

In [13]:
batch = make_batch()

In [14]:
#len of batch
len(batch)

16

In [15]:
#we can deconstruct using map and zip
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))
input_ids.shape, segment_ids.shape, masked_tokens.shape, masked_pos.shape, isNext.shape

(torch.Size([16, 128]),
 torch.Size([16, 128]),
 torch.Size([16, 10]),
 torch.Size([16, 10]),
 torch.Size([16]))

## 3. Model

Recall that BERT only uses the encoder.

BERT has the following components:

- Embedding layers
- Attention Mask
- Encoder layer
- Multi-head attention
- Scaled dot product attention
- Position-wise feed-forward network
- BERT (assembling all the components)

## 3.1 Embedding

Here we simply generate the positional embedding, and sum the token embedding, positional embedding, and segment embedding together.

In [16]:
class Embedding(nn.Module):
    def __init__(self, vocab_size, max_len, n_segments, d_model, device):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(max_len, d_model)      # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)
        self.device = device

    def forward(self, x, seg):
        #x, seg: (bs, len)
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long).to(self.device)
        pos = pos.unsqueeze(0).expand_as(x)  # (len,) -> (bs, len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

## 3.2 Attention mask

In [17]:
def get_attn_pad_mask(seq_q, seq_k, device):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1).to(device)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

### Testing the attention mask

In [18]:
print(get_attn_pad_mask(input_ids, input_ids, device).shape)

torch.Size([16, 128, 128])


In [19]:
print(get_attn_pad_mask(input_ids, input_ids, device).shape)

torch.Size([16, 128, 128])


## 3.3 Encoder

The encoder has two main components: 

- Multi-head Attention
- Position-wise feed-forward network

First let's make the wrapper called `EncoderLayer`

In [20]:
class EncoderLayer(nn.Module):
    def __init__(self, n_heads, d_model, d_ff, d_k, device):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention(n_heads, d_model, d_k, device)
        self.pos_ffn       = PoswiseFeedForwardNet(d_model, d_ff)

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

Let's define the scaled dot attention, to be used inside the multihead attention

In [21]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k, device):
        super(ScaledDotProductAttention, self).__init__()
        self.scale = torch.sqrt(torch.FloatTensor([d_k])).to(device)

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / self.scale # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn 

Here is the Multiheadattention.

In [22]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, d_model, d_k, device):
        super(MultiHeadAttention, self).__init__()
        self.n_heads = n_heads
        self.d_model = d_model
        self.d_k = d_k
        self.d_v = d_k
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, self.d_v * n_heads)
        self.device = device
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, self.n_heads, self.d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention(self.d_k, self.device)(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = nn.Linear(self.n_heads * self.d_v, self.d_model, device=self.device)(context)
        return nn.LayerNorm(self.d_model, device=self.device)(output + residual), attn # output: [batch_size x len_q x d_model]

Here is the PoswiseFeedForwardNet.

In [23]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(F.gelu(self.fc1(x)))

## 3.4 Putting them together

In [24]:
class BERT(nn.Module):
    def __init__(self, n_layers, n_heads, d_model, d_ff, d_k, n_segments, vocab_size, max_len, device):
        super(BERT, self).__init__()
        self.params = {'n_layers': n_layers, 'n_heads': n_heads, 'd_model': d_model,
                       'd_ff': d_ff, 'd_k': d_k, 'n_segments': n_segments,
                       'vocab_size': vocab_size, 'max_len': max_len}
        self.embedding = Embedding(vocab_size, max_len, n_segments, d_model, device)
        self.layers = nn.ModuleList([EncoderLayer(n_heads, d_model, d_ff, d_k, device) for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))
        self.device = device

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids, self.device)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]
        
        # 1. predict next sentence
        # it will be decided by first token(CLS)
        h_pooled   = self.activ(self.fc(output[:, 0])) # [batch_size, d_model]
        logits_nsp = self.classifier(h_pooled) # [batch_size, 2]

        # 2. predict the masked token
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked  = self.norm(F.gelu(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_nsp
    
    def get_last_hidden_state(self, input_ids, segment_ids):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids, self.device)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)

        return output

## 4. Training

In [25]:
from tqdm.auto import tqdm


# BERT configuration — THESE are the actual training params

n_layers = 6
n_heads  = 8
d_model  = 512
d_ff     = d_model * 4   # 2048
d_k = d_v = d_model // n_heads  # 64
n_segments = 2

num_epoch = 15            # more epochs for better learning
num_batches_per_epoch = 50  # more batches per epoch (was 20)

model = BERT(
    n_layers, n_heads, d_model, d_ff, d_k,
    n_segments, vocab_size, max_len, device
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)  # slightly lower lr for stability

print(f"Config: n_layers={n_layers}, n_heads={n_heads}, d_model={d_model}, d_ff={d_ff}, d_k={d_k}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Training: {num_epoch} epochs × {num_batches_per_epoch} batches × {batch_size} samples = {num_epoch * num_batches_per_epoch * batch_size:,} total samples")

Config: n_layers=6, n_heads=8, d_model=512, d_ff=2048, d_k=64
Model parameters: 29,754,910
Training: 15 epochs × 50 batches × 16 samples = 12,000 total samples


In [26]:
import time
t0 = time.time()
_ = make_batch()
print("make_batch seconds:", time.time()-t0)


make_batch seconds: 16.232064485549927


In [27]:
from tqdm.auto import tqdm

total_steps = num_epoch * num_batches_per_epoch
train_bar = tqdm(total=total_steps, desc="Training Progress", unit="step")

for epoch in range(num_epoch):
    model.train()
    epoch_loss = 0.0

    # Show batch-building progress
    build_bar = tqdm(range(num_batches_per_epoch),
                     desc=f"Building Batches (Epoch {epoch+1})",
                     leave=False)

    cached_batches = []
    for _ in build_bar:
        cached_batches.append(make_batch())

    build_bar.close()

    # Now training steps update immediately
    for batch in cached_batches:
        input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

        input_ids = input_ids.to(device)
        segment_ids = segment_ids.to(device)
        masked_tokens = masked_tokens.to(device)
        masked_pos = masked_pos.to(device)
        isNext = isNext.to(device)

        optimizer.zero_grad(set_to_none=True)
        logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos)

        loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens).float().mean()
        loss_nsp = criterion(logits_nsp, isNext)
        loss = loss_lm + loss_nsp

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        train_bar.update(1)

    print(f"Epoch {epoch+1}/{num_epoch} avg_loss = {epoch_loss/num_batches_per_epoch:.6f}")

train_bar.close()
print("Training complete!")


Training Progress:   0%|          | 0/750 [00:00<?, ?step/s]

Building Batches (Epoch 1):   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/15 avg_loss = 37.317722


Building Batches (Epoch 2):   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 2/15 avg_loss = 18.370812


...

Epoch 13/15 avg_loss = 4.880056


Building Batches (Epoch 14):   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 14/15 avg_loss = 4.975301


Building Batches (Epoch 15):   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 15/15 avg_loss = 5.004114
Training complete!


In [48]:
# Save the model after training
torch.save(model.state_dict(), 'bert_model.pth')
print("Model saved to bert_model.pth")

Model saved to bert_model.pth


## 5. Inference

Since our dataset is relatively small (100k samples from BookCorpus), the model won't perform as well as the full BERT, but it demonstrates the training process. The saved weights (`bert_model.pth`) will be used in Task 2 for fine-tuning on downstream tasks.

In [28]:
# Create a fresh batch for inference testing
test_batch = make_batch()

# Predict mask tokens and isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(test_batch[0]))
print("Input sentence tokens:")
print([id2word[w.item()] for w in input_ids[0] if id2word[w.item()] != '[PAD]'])

input_ids = input_ids.to(device)
segment_ids = segment_ids.to(device)
masked_tokens = masked_tokens.to(device)
masked_pos = masked_pos.to(device)
isNext = isNext.to(device)

logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos)
# logits_lm:  (1, max_mask, vocab_size)
# logits_nsp: (1, 2)

# predict masked tokens
# max the probability along the vocab dim (2), [1] is the indices of the max
logits_lm = logits_lm.data.cpu().max(2)[1][0].data.numpy()

print('masked tokens (ground truth) : ', [id2word[pos.item()] for pos in masked_tokens[0]])
print('predict masked tokens :        ', [id2word[pos.item()] for pos in logits_lm])

# predict nsp
logits_nsp = logits_nsp.cpu().data.max(1)[1][0].data.numpy()
print('isNext (ground truth) : ', True if isNext else False)
print('predict isNext :        ', True if logits_nsp else False)

Input sentence tokens:
['[CLS]', 'are', 'we', 'one', '[MASK]', 'those', 'couples', '[SEP]', '[MASK]', 'will', 'make', 'it', 'so', 'much', 'easier', "''", '[SEP]']
masked tokens (ground truth) :  ['that', 'those', 'of', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
predict masked tokens :         ['i', 'i', 'i', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
isNext (ground truth) :  False
predict isNext :         False


save model with configuration 

In [29]:
# Save model + config + vocab (all consistent)
import json

torch.save({
    "state_dict": model.state_dict(),
    "config": {
        "n_layers": n_layers,
        "n_heads": n_heads,
        "d_model": d_model,
        "d_ff": d_ff,
        "d_k": d_k,
        "n_segments": n_segments,
        "vocab_size": vocab_size,
        "max_len": max_len
    }
}, "bert_model_with_config.pth")

with open('word2id.json', 'w') as f:
    json.dump(word2id, f)

# Verify
ckpt = torch.load('bert_model_with_config.pth', map_location='cpu')
print("Saved config:", ckpt['config'])
print("tok_embed shape:", ckpt['state_dict']['embedding.tok_embed.weight'].shape)
print(f"Vocab saved: {len(word2id)} words")

# Also save classifier-compatible info
print(f"\nFor Task 2: d_model*3 = {d_model*3} (classifier input size)")

Saved config: {'n_layers': 6, 'n_heads': 8, 'd_model': 512, 'd_ff': 2048, 'd_k': 64, 'n_segments': 2, 'vocab_size': 23068, 'max_len': 128}
tok_embed shape: torch.Size([23068, 512])
Vocab saved: 23068 words

For Task 2: d_model*3 = 1536 (classifier input size)
